In [1]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
conn = sqlite3.connect(r'C:\Users\Spencer\Environments\aquastaat\project.db')
conn.text_factory = lambda x: str(x, 'latin1')

water = pd.read_sql("SELECT * FROM parameter", conn)

filled_efr = pd.read_csv(r'data\table\filledEFR.csv')
filled_efr = filled_efr.loc[:, ['Area_x', 'EFR']]
filled_efr.columns = ['Area', 'EFR']
filled_efr.set_index('Area', inplace=True)

In [3]:
All = slice(None)

pop = 'Total population'
cult = 'Cultivated area (arable land + permanent crops)'
gdp = 'Gross Domestic Product (GDP)'
trsw = 'Total renewable surface water'
tfw = 'Total freshwater withdrawal (primary and secondary)'
trwr = 'Total renewable water resources'
efr = 'Environmental Flow Requirements'
wsi = 'SDG 6.4.2. Water Stress'

preds = [cult, gdp, pop, trsw]

trwr_id = 4188
efr_id = 4549

In [4]:
data = pd.read_csv('training_data_arima.csv')
df_model = data.copy()
data.set_index(['Area', 'YearBin'], inplace=True)

In [5]:
# df_model.columns = df_model.columns.droplevel()
df_model.rename(columns={'Cultivated area (arable land + permanent crops)':'cultivated_area', 'Gross Domestic Product (GDP)':'GDP', 'Total freshwater withdrawal (primary and secondary)':'TFW', 'Total population':'total_population', 'Total renewable surface water':'TRSW'}, inplace=True)
df_model.rename_axis(None, inplace=True)

# Remove rows that contains NaN
df_model.dropna(inplace=True)
print(len(df_model.index))

1336


In [6]:
x_data = df_model[['cultivated_area', 'GDP', 'total_population', 'TRSW',]]
y_data = df_model['TFW']

## One-hot encoding
# enc = OneHotEncoder(handle_unknown='ignore')
# x_data = enc.fit_transform(x_data)

## Split data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.75, random_state=100, shuffle=True)

## Build a random forest regressor
random = 100
RF_clf = RandomForestRegressor(bootstrap=True, criterion='mse',random_state=random)
RF_clf.fit(x_train, y_train)
RF_param_grid = {"n_estimators":[10, 20, 30,  50,100], "max_depth": [5, 10, 15, 20, None]}

## Optimize the randomForest regressor
RF_grid_search = GridSearchCV(RF_clf, param_grid=RF_param_grid, n_jobs=-1, cv=3)
RF_grid_search.fit(x_train, y_train)
print(RF_grid_search.best_params_)
print(RF_grid_search.best_score_)

## Define optimum randomForest regressor
opt_RF = RandomForestRegressor(bootstrap=True, criterion='mse',random_state=random, n_estimators=RF_grid_search.best_params_['n_estimators'], max_depth=RF_grid_search.best_params_['max_depth'])
opt_RF.fit(x_train, y_train)
RF_tr_pred = opt_RF.predict(x_train)
print("Training MSE: %.2f" %mean_squared_error(y_train, RF_tr_pred))
RF_test_pred = opt_RF.predict(x_test)
print("Test MSE: %.2f" %mean_squared_error(y_test, RF_test_pred))

## Calculate adjusted R-squared
# n1 = number of data points for training set
# n2 = number of data points for test set
# p = number of predictors / attributes
n1 = len(y_train)
p=4
n2 = len(y_test)
train_adj_r2 = 1-(1-r2_score(y_train, RF_tr_pred))*(n1-1)/(n1-p-1)
test_adj_r2 = 1-(1-r2_score(y_test, RF_test_pred))*(n2-1)/(n2-p-1)
print("Training adj R2: %.2f" %train_adj_r2) 
print("Test adj R2: %.2f" %test_adj_r2) 

c:\users\spencer\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'max_depth': 15, 'n_estimators': 30}
0.9385050033887423
Training MSE: 31.90
Test MSE: 284.07
Training adj R2: 0.99
Test adj R2: 0.95


In [7]:
tfw_preds = opt_RF.predict(data.loc[(All, [2020, 2025, 2030]), preds].values)
data.loc[(All, [2020, 2025, 2030]), tfw] = tfw_preds

In [8]:
water.set_index(['Area', 'VariableId'], inplace=True)
water.sort_index(inplace=True)
water = water.loc[(All, [trwr_id, efr_id]), ['Value']]
water.fillna(0, inplace=True)
water.loc[water.loc[:,'Value']=='', :] = 0

In [9]:
data[trwr] = None
data[efr] = None
for name, group in data.groupby('Area'):
    if (name, trwr_id) in water.index:
        data.loc[(name, All), trwr] = water.at[(name, trwr_id), 'Value'].max()
    if name in filled_efr.index:
        data.loc[(name, All), efr] = filled_efr.at[name, 'EFR']

In [10]:
data[wsi] = 100*data[tfw]/(data[trwr] - data[efr])

In [11]:
data.loc[('Kuwait', All), :]

Total population  \
Area   YearBin                     
Kuwait 1980         14104.000000   
       1985          1943.000000   
       1990          1917.000000   
       1995          1715.000000   
       2000          2144.000000   
       2005          2503.000000   
       2010          3396.000000   
       2015          4137.000000   
       2020          4518.536071   
       2025          4935.259422   
       2030          5390.415208   

                Cultivated area (arable land + permanent crops)  \
Area   YearBin                                                    
Kuwait 1980                                            2.000000   
       1985                                            4.000000   
       1990                                            5.000000   
       1995                                            7.000000   
       2000                                           15.000000   
       2005                                           15.000000   
       2010                                           16.000000   
       2015                                           14.000000   
       2020                                           15.181818   
       2025                                           16.363636   
       2030                                           17.545455   

                Gross Domestic Product (GDP)  Total renewable surface water  \
Area   YearBin                                                                
Kuwait 1980                     2.160000e+10                            0.0   
       1985                     2.240000e+10                            0.0   
       1990                     1.990000e+10                            0.0   
       1995                     3.040000e+10                            0.0   
       2000                     3.810000e+10                            0.0   
       2005                     1.150000e+11                            0.0   
       2010                     1.740000e+11                            0.0   
       2015                     1.130000e+11                            0.0   
       2020                     1.229372e+11                            0.0   
       2025                     1.337482e+11                            0.0   
       2030                     1.455100e+11                            0.0   

               SDG 6.4.2. Water Stress  \
Area   YearBin                           
Kuwait 1980                       1275   
       1985                       1275   
       1990                       1275   
       1995                       1275   
       2000                       2075   
       2005                       2075   
       2010                       2075   
       2015                       2075   
       2020                    7514.16   
       2025                    9071.37   
       2030                    9295.81   

                Total freshwater withdrawal (primary and secondary)  \
Area   YearBin                                                        
Kuwait 1980                                              0.255000     
       1985                                              0.255000     
       1990                                              0.255000     
       1995                                              0.255000     
       2000                                              0.415000     
       2005                                              0.415000     
       2010                                              0.415000     
       2015                                              0.415000     
       2020                                              1.502831     
       2025                                              1.814275     
       2030                                              1.859161     

               Total renewable water resources Environmental Flow Requirements  
Area   YearBin                                                                  
Kuwait 1980   

In [12]:
data.dropna(axis=0, inplace=True)

In [13]:
view = (group.loc[:, [tfw]] for name, group in data.groupby('Area'))

In [14]:
next(view)

Total freshwater withdrawal (primary and secondary)
Area        YearBin                                                     
Afghanistan 1980                                             26.110000  
            1985                                             26.110000  
            1990                                             24.166667  
            1995                                             22.223333  
            2000                                             20.280000  
            2005                                             20.280000  
            2010                                             20.280000  
            2015                                             20.280000  
            2020                                             10.788403  
            2025                                             15.042920  
            2030                                             24.605739

In [15]:
data.to_csv('output_gdp0_pop0.csv')

Save model

In [ ]:
import pickle

In [ ]:
pickle.dump(opt_RF, open('random_forest.sav', 'wb'))